In [16]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_google_repos_final():
    """
    GitHub上でGoogleが管理するリポジトリの情報を取得します。
    time.sleep(1)を遵守し、全ページをスクレイピングするフレームワークです。
    """
    
    # 課題の条件を満たすための設定
    BASE_URL = 'https://github.com/orgs/google/repositories'
    BASE_QUERY = '?q=&type=all&language=&sort=stargazers' 
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    all_repo_data = []
    page = 1
    MAX_PAGES = 500 # 安全装置

    # --- データ取得ループ ---
    while page <= MAX_PAGES: 
        current_url = f"{BASE_URL}{BASE_QUERY}&page={page}"
        print(f"ページ {page} ({current_url}) をスクレイピング中...")

        try:
            res = requests.get(current_url, headers=HEADERS)
            res.raise_for_status() 

            soup = BeautifulSoup(res.text, 'html.parser')
            main_container = soup.find('div', id='org-repositories')
            repo_list = main_container.select('li') if main_container else []
            
            valid_repos_on_page = 0
            
            # --- 抽出ロジック ---
            for repo in repo_list: 
                repo_info = {}
                name_tag = repo.find('a', class_='text-bold')
                if not name_tag: continue
                    
                repo_info['リポジトリ名'] = name_tag.text.strip()
                language_tag = repo.find('span', itemprop='programmingLanguage')
                repo_info['主要な言語'] = language_tag.text.strip() if language_tag else 'N/A'
                
                star_tag = repo.find('a', href=lambda h: h and 'stargazers' in h)
                star_count_text = star_tag.get_text(strip=True) if star_tag else '0'

                star_str = star_count_text.lower().replace(',', '')
                star_count = 0
                
                if 'k' in star_str:
                    star_count = int(float(star_str.replace('k', '')) * 1000)
                elif 'm' in star_str:
                    star_count = int(float(star_str.replace('m', '')) * 1000000)
                else:
                    try:
                        star_count = int(star_str)
                    except:
                        star_count = 0
                        
                repo_info['スターの数'] = star_count
                all_repo_data.append(repo_info)
                valid_repos_on_page += 1
            
            print(f"ページ {page} から {valid_repos_on_page} 件のリポジトリを抽出しました。")
            
            # 終了条件
            if valid_repos_on_page == 0:
                print("リポジトリが一つも抽出されませんでした。最終ページと判断しスクレイピングを終了します。")
                break
            
            # 課題条件: time.sleep(1) の挿入
            print("サーバー保護のため1秒待機します。")
            time.sleep(1)
            
            page += 1

        except requests.exceptions.RequestException as e:
            print(f"エラー: リクエスト失敗によりスクレイピングを中断しました。")
            break
            
    # --- 結果の表示 ---
    unique_repos = {repo['リポジトリ名']: repo for repo in all_repo_data}.values()
    unique_repos_list = list(unique_repos)
    
    if page > MAX_PAGES:
         print("\n注意: 設定された最大ページ数に達したため、強制的にスクレイピングを終了しました。")

    print(f"\n========================================================")
    print(f"全ページスクレイピング完了。合計 {len(unique_repos_list)} 件のユニークなリポジトリを取得しました。")
    print("========================================================")
    
    # スターの多い順にソート
    repo_data_sorted = sorted(unique_repos_list, key=lambda x: x['スターの数'], reverse=True)
    
    print("\n--- 取得結果 ---")
    print("| リポジトリ名 | 主要な言語 | スターの数 |")
    print("| :--- | :--- | :--- |")
    
    for item in repo_data_sorted:
        star_str = f"{item['スターの数']:,}"
        print(f"| {item['リポジトリ名']} | {item['主要な言語']} | {star_str} |")

    return all_repo_data

# スクリプトの実行
scraped_data = scrape_google_repos_final()

ページ 1 (https://github.com/orgs/google/repositories?q=&type=all&language=&sort=stargazers&page=1) をスクレイピング中...
ページ 1 から 0 件のリポジトリを抽出しました。
リポジトリが一つも抽出されませんでした。最終ページと判断しスクレイピングを終了します。

全ページスクレイピング完了。合計 0 件のユニークなリポジトリを取得しました。

--- 取得結果 ---
| リポジトリ名 | 主要な言語 | スターの数 |
| :--- | :--- | :--- |


### ダミーデータでDBに保存するコード例

In [18]:
import sqlite3

path = ''
db_name = 'assignment.db'

try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()

    # 1. 既存のテーブルを削除する (重要！)
    cur.execute('DROP TABLE IF EXISTS assignment;')

    # 2. 正しいテーブル構造で再作成
    # titleとlanguageはTEXT型、starsはINT型
    sql = 'CREATE TABLE assignment(title TEXT, language TEXT, stars INT);'

    cur.execute(sql)
    conn.commit()
    print("テーブルの再作成が完了しました。")

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:    
    if 'conn' in locals() and conn:
        conn.close()

テーブルの再作成が完了しました。


In [19]:
path = ''
db_name = 'assignment.db'

try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()

    sql = "INSERT INTO assignment (title, language, stars) VALUES (?, ?, ?);"

    assignment_dict = [
        {"title": "dummy-repo-tensorflow", "language": "Python", "stars": 180000},
        {"title": "dummy-repo-flutter", "language": "Dart", "stars": 160000},
        {"title": "dummy-repo-go", "language": "Go", "stars": 123456},
        {"title": "dummy-repo-skia", "language": "C++", "stars": 8000},
        {"title": "dummy-repo-chrome", "language": "C++", "stars": 50},
    ]
    
    assignment_data = [
        (r['title'], r['language'], r['stars']) for r in assignment_dict
    ]

    cur.executemany(sql, assignment_data)
    conn.commit()
    print("データの挿入完了。")

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally: 
    if 'conn' in locals() and conn:
        conn.close()

データの挿入完了。


In [20]:
path = ''
db_name = 'assignment.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()

    # データを参照するSQL文の作成
    sql = "SELECT title, language, stars FROM assignment ORDER BY stars DESC;"

    # SQL文の実行
    cur.execute(sql)

except sqlite3.Error as e:
    print('エラーが発生しました:', e)
    
else:
    # SELECT結果をきれいに表示
    print("\n--- DBからSELECTした結果 ---")
    print("| {:<20} | {:<10} | {:>8} |".format("タイトル", "言語", "スター数"))
    print("|" + "-"*22 + "|" + "-"*12 + "|" + "-"*10 + "|")
    
    for row in cur:
        # 取得した列の順番と変数が一致するため、エラーにならない
        title, language, stars = row
        
        # スター数をカンマ区切りで整形して表示
        print("| {:<20} | {:<10} | {:>8,} |".format(title, language, stars))


finally: 
    # DBへの接続を閉じる
    if 'conn' in locals() and conn:
        conn.close()


--- DBからSELECTした結果 ---
| タイトル                 | 言語         |     スター数 |
|----------------------|------------|----------|
| dummy-repo-tensorflow | Python     |  180,000 |
| dummy-repo-flutter   | Dart       |  160,000 |
| dummy-repo-go        | Go         |  123,456 |
| dummy-repo-skia      | C++        |    8,000 |
| dummy-repo-chrome    | C++        |       50 |
